In [1]:
from os.path import join, split
import torch 
import numpy as np
import pandas as pd
from common import metrics
from common.metrics import pr_auc, roc_auc, precision, recall
from common.io import get_train_test_for_embeddings
import common.io as io
import importlib 
importlib.reload(io)
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate, ParameterGrid
from imblearn.over_sampling import RandomOverSampler
import importlib
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from cope import model

from tqdm import tqdm

/services/tools/anaconda3/4.4.0/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [2]:
data_path = join('runs', 'pretraining', 'embeddings', 'censored')

## Train classifiers on Embeddings 

In [3]:
# X, y, X_test, y_test = io.get_train_test_for_embeddings(data_path,  'HOSPITAL_ADMISSION', 'DEATH', True, filter_covid=True) 
# X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=.2)
# len(X_train)

In [4]:
# clf = RandomForestClassifier(n_estimators=30, oob_score=True)
# ros = RandomOverSampler(random_state=0, sampling_strategy=.3)
# X_resampled, y_resampled = ros.fit_resample(X_test, y_test,)
# clf.fit(X_resampled[:3000], y_resampled[:3000])
# print(pr_auc(clf, X_val, y_val))
# print(roc_auc(clf, X_val, y_val))

## RF CV results

In [23]:
def get_closest_ids(arr, value):
    idx = (np.abs(arr - value)).argmin()
    return idx

In [27]:
Results_dic = {}
CM_dic = {}
n_folds = 5
sensitivity_threshold = 0.5
# threshold = .1
sampling_alpha = .3
oversample=False
metric_ls = [roc_auc, pr_auc, precision, recall]# 
metric_names = [m.__name__ for m in metric_ls]
results_dir = "runs/pretraining/embeddings/censored/Results"
# parameter_grid = {'n_estimators':[40],'max_depth':[5,10, None], 'min_samples_split':[2,5,10]}
parameter_grid = {'n_estimators':[1000],'max_depth':[5,10, None], 'min_samples_split':[2,5 ,10]}

probas_dic = {}
for censor in ['HOSPITAL_ADMISSION', 'ICU_ADMISSION']:
    for post in [False, True]:
        suffix='pre'
        if post:
            suffix='post'    
        if not post and (censor=='HOSPITAL_ADMISSION'):
            continue
        for target in ['ICU_ADMISSION', 'MECHANICAL_VENTILATION','DEATH']:
            if censor==target:
                continue
            print(f"censor {suffix} {censor} target {target}")
            name = f'{suffix}_{censor}_target_{target}'
            CM_dic[name] = []
            probas_dic[name] = []
            _, _, X_test, y_test = io.get_train_test_for_embeddings(data_path, censor, target, post, filter_covid=True)   
            scores = {metric_name:[] for metric_name in metric_names}
            skf = StratifiedKFold(n_folds)
            for i, fold in tqdm(enumerate(skf.split(X_test,y_test)), desc='CV'):
                train_ids, val_ids = fold
                X_train, X_val, y_train, y_val = X_test[train_ids], X_test[val_ids], y_test[train_ids], y_test[val_ids]
                if oversample:
                    ros = RandomOverSampler(random_state=0, sampling_strategy=sampling_alpha)
                    X_train, y_train = ros.fit_resample(X_train, y_train)

                rf = model.RF(name)
                rf.fit(X_train, y_train, parameter_grid)
                
                # save curves
                y_scores = rf._model.predict_proba(X_val)[:,1]
                
                precisions, recalls, th_pr = precision_recall_curve(y_val, y_scores)
                
                pr_df = pd.DataFrame({'precision':precisions[:-1], 'recall':recalls[:-1], 'thresholds':th_pr})
                fprs, tprs, th_roc = roc_curve(y_val, y_scores)
                
                
                idx = get_closest_ids(tprs, sensitivity_threshold)
                cm_threshold_roc = th_roc[idx]
                idx = get_closest_ids(recalls, sensitivity_threshold)
                cm_threshold_pr = th_pr[idx]
                cm_threshold = (cm_threshold_pr+cm_threshold_roc)/2
                
                roc_df = pd.DataFrame({'fpr':fprs, 'tpr':tprs, 'thresholds':th_roc})
                pr_df.to_csv(join(results_dir,'curves',f"pr_curve_{name}_avg_emb_{i}.csv"),index=None)
                roc_df.to_csv(join(results_dir,'curves',f"roc_curve_{name}_avg_emb_{i}.csv"),index=None)
#                 predictions = rf._model.predict(X_test)
                probas = rf._model.predict_proba(X_val)[:,1]
                probas_dic[name].append(probas)
                predictions = (probas > cm_threshold).astype(int)
                cm = confusion_matrix(y_val, predictions, labels=[0, 1])
                CM_dic[name].append(cm)
                for metric in metric_ls:
                    scores[metric.__name__].append(metric(rf._model, X_val, y_val))
                    
            Results_dic[name] = scores

torch.save(CM_dic, f"runs/pretraining/embeddings/censored/Results/cm_mean_{sensitivity_threshold}_sensitivity.pt")
torch.save(probas_dic, f"runs/pretraining/embeddings/censored/Results/probas_mean.pt")

censor post HOSPITAL_ADMISSION target ICU_ADMISSION



CV: 0it [00:00, ?it/s]

Selected thresholds at 0.5 sensitivity:  0.10053516405904264 0.11005683587007901


/services/tools/anaconda3/4.4.0/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

CV: 1it [00:37, 37.14s/it]

Selected thresholds at 0.5 sensitivity:  0.1286650649816331 0.1304563002471126


...


CV: 3it [02:01, 40.62s/it]

Selected thresholds at 0.5 sensitivity:  0.3762300278797571 0.3896643133225565



CV: 4it [02:42, 40.72s/it]

Selected thresholds at 0.5 sensitivity:  0.35499605258244116 0.3569895403047949


CV: 5it [03:22, 40.47s/it]


In [ ]:
def get_mean_std(Results_dic):
    """Takes a nested dict with methods as outer dict and metrics as inner dict which contains lists os metrics,
    and computes mean \pm std for each list."""
    Results_mean_std = {}

    for method in Results_dic.keys():
        Results_mean_std[method] = {}
        for metric in Results_dic[method].keys():
            mean = np.mean(np.array(Results_dic[method][metric]))
            std = np.std(np.array(Results_dic[method][metric]))
            Results_mean_std[method][metric] = f'{mean:.3f} ± {std:.3f}'
    return Results_mean_std

In [ ]:
Results_str = get_mean_std(Results_dic)

In [ ]:
# COVID paper results
Results_org = {
    "post_HOSPITAL_ADMISSION_target_ICU_ADMISSION":{"org_roc_auc":"0.752", "org_pr_auc":"0.282"},
    "post_HOSPITAL_ADMISSION_target_MECHANICAL_VENTILATION":{"org_roc_auc":"0.743", "org_pr_auc":"0.238"},
    "post_HOSPITAL_ADMISSION_target_DEATH":{"org_roc_auc":"0.794", "org_pr_auc":"0.445"},
    "pre_ICU_ADMISSION_target_MECHANICAL_VENTILATION":{"org_roc_auc":"0.567", "org_pr_auc":"0.869"},
    "pre_ICU_ADMISSION_target_DEATH":{"org_roc_auc":"0.735", "org_pr_auc":"0.548"},
    "post_ICU_ADMISSION_target_MECHANICAL_VENTILATION":{"org_roc_auc":"0.530", "org_pr_auc":"0.843"},
    "post_ICU_ADMISSION_target_DEATH":{"org_roc_auc":"0.724", "org_pr_auc":"0.552"},
}

In [ ]:
All_Results = {key:{**Results_str[key], **Results_org[key]} for key in Results_str}
results = pd.DataFrame(All_Results).T
# results = results[results.index!='ICU_ADMISSION_post_target_ICU_ADMISSION']
results

In [ ]:
results.to_csv("runs/pretraining/embeddings/censored/Results/mean.csv")